In [1]:
 from __future__ import print_function 

 import keras
 from keras.datasets import mnist
 from keras.models import Model
 from keras.layers import Input,Dense,TimeDistributed
 from keras.layers import LSTM


In [2]:
# Training parameters.
batch_size = 32
num_classes = 10
epochs = 5

In [4]:
#Embedding dimensions.
row_hidden = 128
col_hidden = 128

In [5]:
# The data, split between train and test sets.
(x_train,y_train),(x_test,y_test) = mnist.load_data() 

11501568/11490434 [==============================] - 0s 0us/step


In [6]:
# Reshape data to 40 for Hierarchical RNN.

#HRNNS can learn accross multiple levels of temporal hierarchy over a complex sequence.Usually,the first recurrent layer of an HRNN encodes a sentence 
# (e.g. of word vectors) into a sentence vector.The second recurrent layer then encodes a sequence of such vectors (encoded by the first layer) into a 
# document vector. This document vector is considered to ppreserve both the word-level and sentence-level structur of the context.

x_train = x_train.reshape(x_train.shape[0],28,28,1)
x_test = x_test.reshape(x_test.shape[0],28,28,1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train.samples')
print(x_test.shape[0], 'test samples')


x_train shape: (60000, 28, 28, 1)
60000 train.samples
10000 test samples


In [8]:
# converts class vectors to binary class matrices.
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train,num_classes) 
y_test = np_utils.to_categorical(y_test, num_classes)

row, col,pixel = x_train.shape[1:]

#40 input 
x= Input(shape=(row,col,pixel))


In [9]:
# Encodes a row of pixels using TimeDistributed Wrapper.
# TimeDistrubuted. This wrapper applies a layer to every temporal slice of an input.The input should be at least 30, and the dimension of index one will be
#considered to be the temporal dimension. Consider a batch of 32 samples, where each sample is a sequence of 10 vectors of 16 dimensions

encoded_rows = TimeDistributed(LSTM(row_hidden)) (x)

#Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)

In [10]:
# Final predictions and model.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [12]:
# Training.
model.fit(x_train,
          y_train,batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test,y_test))

Epoch 1/5
1875/1875 [==============================] - 64s 30ms/step - loss: 0.4502 - accuracy: 0.8494 - val_loss: 0.1680 - val_accuracy: 0.9518
Epoch 2/5
1875/1875 [==============================] - 55s 29ms/step - loss: 0.1093 - accuracy: 0.9671 - val_loss: 0.0813 - val_accuracy: 0.9764
Epoch 3/5
1875/1875 [==============================] - 57s 31ms/step - loss: 0.0761 - accuracy: 0.9772 - val_loss: 0.0670 - val_accuracy: 0.9795
Epoch 4/5
1875/1875 [==============================] - 55s 29ms/step - loss: 0.0607 - accuracy: 0.9818 - val_loss: 0.0613 - val_accuracy: 0.9819
Epoch 5/5
1875/1875 [==============================] - 55s 29ms/step - loss: 0.0494 - accuracy: 0.9850 - val_loss: 0.0979 - val_accuracy: 0.9726


In [13]:
#Evaluation
scores = model.evaluate(x_test,y_test,verbose=0)
print('Test loss:',scores[0])
print('Test accuracy:',scores[1])

Test loss: 0.09785681962966919
Test accuracy: 0.972599983215332
